# Gene Expression Analysis of Project TCGA-CHOL

*Please note: This notebook uses open access data*


### Qiong Liu
### April 7th, 2022

Cholangiocarcinoma (CCA) is aggressive cancer found in the slender tubes that carry the digestive fluid bile through the liver. The Cancer Genome Atlas (TCGA) program contains abundant molecular profilings of over 20,000 primary cancer and matched normal samples spanning 33 cancer types. In this notebook, we demonstrated how to retrieve RNA expression data of project TCGA-CHOL from [Genomic Data Commons (GDC) data portal](https://portal.gdc.cancer.gov/), and perform data analysis and visualization using a pipeline provided by an R package `GDCRNATools`. 

This pipeline was modified based on the manual of [GDCRNATools](http://bioconductor.org/packages/devel/bioc/vignettes/GDCRNATools/inst/doc/GDCRNATools.html).

**References**

- Li R, Qu H, Wang S, Wei J, Le Zhang, Ma R, Lu J, Zhu J, Zhong W, Jia Z (2021). GDCRNATools: GDCRNATools: an R/Bioconductor package for integrative analysis of lncRNA, mRNA, and miRNA data in GDC. https://doi.org/10.1093/bioinformatics/bty124

- Love, M.I., Huber, W. & Anders, S. Moderated estimation of fold change and dispersion for RNA-seq data with DESeq2. Genome Biol 15, 550 (2014). https://doi.org/10.1186/s13059-014-0550-8

- Ritchie, M. E., Phipson, B., Wu, D., Hu, Y., Law, C. W., Shi, W., & Smyth, G. K. (2015). limma powers differential expression analyses for RNA-sequencing and microarray studies. Nucleic acids research, 43(7), e47. https://doi.org/10.1093/nar/gkv007

- Robinson, M. D., McCarthy, D. J., & Smyth, G. K. (2010). edgeR: a Bioconductor package for differential expression analysis of digital gene expression data. Bioinformatics (Oxford, England), 26(1), 139–140. https://doi.org/10.1093/bioinformatics/btp616

---


## Contents

- [Data preparation](#Data-Preparation)
- [Differential expression analysis](#Differential-expression-analysis)
- [Functional enrichment analysis](#Functional-enrichment-analysis)
- [Univariate survival analysis](#Univariate-survival-analysis)


## Data Preparation

### Import R packages

In [ ]:
# suppress warnings (to include warnings,set warn to 0)
options(warn=-1)

library(GDCRNATools)
library(dplyr)
library(tidyr)

### Download GDC data transfer tool

The R package of `GDCRNATools` uses `gdc-client` data transfer to download the object file. Run the command below to download and unzip gdc-client.

In [ ]:
system("wget https://gdc.cancer.gov/files/public/file/gdc-client_v1.6.1_Ubuntu_x64.zip")
unzip("gdc-client_v1.6.1_Ubuntu_x64.zip", exdir=".")

### Data download

- The block blow fixes a bug in gdcGetURL that leads to downloading error, and overwrites the orginal function `gdcGetURL` within the pakcage namespace.

In [ ]:
gdcGetURL_new <- function(project.id, data.type) {
    urlAPI <- 'https://api.gdc.cancer.gov/files?'
    
    if (data.type=='RNAseq') {
        data.category <- 'Transcriptome Profiling'
        data.type <- 'Gene Expression Quantification'
        workflow.type <- 'STAR - Counts'
    } else if (data.type=='miRNAs') {
        data.category <- 'Transcriptome Profiling'
        data.type <- 'Isoform Expression Quantification'
        workflow.type <- 'BCGSC miRNA Profiling'
    } else if (data.type=='Clinical') {
        data.category <- 'Clinical'
        data.type <- 'Clinical Supplement'
        workflow.type <- NA
    } else if (data.type=='pre-miRNAs') {
        data.category <- 'Transcriptome Profiling'
        data.type <- 'miRNA Expression Quantification'
        workflow.type <- 'BCGSC miRNA Profiling'
    }
    
    project <- paste('{"op":"in","content":{"field":"cases.',
        'project.project_id","value":["', 
        project.id, '"]}}', sep='')
    dataCategory <- paste('{"op":"in","content":{"field":"files.', 
        'data_category","value":"', data.category, '"}}', sep='')
    dataType <- paste('{"op":"in","content":{"field":"files.data_type",',
        '"value":"', data.type, '"}}', sep='')
    workflowType <- paste('{"op":"in","content":{"field":"files.',
        'analysis.workflow_type","value":"', workflow.type, '"}}', sep='')
    
    
    if (is.na(workflow.type)) {
        dataFormat <- paste('{"op":"in","content":{"field":"files.',
            'data_format","value":"', 'BCR XML', '"}}', sep='')
        content <- paste(project, dataCategory, dataType, dataFormat, sep=',')
    } else {
        content <- paste(project, dataCategory, dataType, 
            workflowType, sep=',')
    }
    
    filters <- paste('filters=',URLencode(paste('{"op":"and","content":[', 
        content, ']}', sep='')),sep='')
    
    expand <- paste('analysis', 'analysis.input_files', 'associated_entities',
        'cases', 'cases.diagnoses','cases.diagnoses.treatments', 
        'cases.demographic', 'cases.project', 'cases.samples', 
        'cases.samples.portions', 'cases.samples.portions.analytes', 
        'cases.samples.portions.analytes.aliquots',
        'cases.samples.portions.slides', sep=',')
    
    expand <- paste('expand=', expand, sep='')
    
    payload <- paste(filters, 'pretty=true', 'format=JSON', 
        'size=10000', expand, sep='&')
    url <- paste(urlAPI, payload, sep='')
    
    return (url)
}

toolenv <- environment(get("gdcGetURL", envir = asNamespace("GDCRNATools")))
unlockBinding("gdcGetURL", toolenv)
assignInNamespace("gdcGetURL", gdcGetURL_new, ns="GDCRNATools", envir=toolenv)
assign("gdcGetURL", gdcGetURL_new)
lockBinding("gdcGetURL", toolenv)


- Download gene expression quantificatioin files of project TCGA-CHOL using gen3-client

In [ ]:
# download RNA-seq quantification files of project TCGA-CHOL
# downloaded files will be stored under TCGA-CHOL_0406/RNAseq folder, respectively
project <- 'TCGA-CHOL_0406'
rnadir <- paste(project, 'RNAseq', sep='/')

# Download RNAseq data 
gdcRNADownload(project.id     = 'TCGA-CHOL', 
               data.type      = 'RNAseq', 
               write.manifest = FALSE,
               method         = 'gdc-client',
               directory      = rnadir)


- Query metadata associated with gene expression quantification files from GDC graph

In [ ]:
# Query metadata(e.g. patient gender, vital status) from GDC graph
# Metadata associated with RNA-seq quantification file
metaMatrix.RNA <- gdcParseMetadata(project.id = 'TCGA-CHOL',
                                   data.type  = 'RNAseq', 
                                   write.meta = FALSE)
# Filter duplicates
metaMatrix.RNA <- gdcFilterDuplicate(metaMatrix.RNA)

# Filter non-Primary Tumor and non-Solid Tissue Normal samples in RNAseq metadata
metaMatrix.RNA <- gdcFilterSampleType(metaMatrix.RNA)


- Inspect first 5 rows of dataframe `metaMatrix.MIR`

In [ ]:
metaMatrix.RNA[1:5,]

- Next two blocks generate `brief summary tables` for several numeric and factor clinical vairables in metaMatrix.RNA (all the patients involved in this notebook)

In [ ]:
# Summary of several numeric variables in metadata
metaMatrix.RNA[,c("age_at_diagnosis", "days_to_death", "days_to_last_follow_up")] %>% summary()

In [ ]:
# Counts of few factor variables in metadata
gender_counts <-metaMatrix.RNA %>% group_by(gender) %>% tally()
sample_type_counts <- metaMatrix.RNA %>% group_by(sample_type) %>% tally()
vital_status_counts <- metaMatrix.RNA %>% group_by(vital_status) %>% tally()
# modify three tables 
gender_counts$category <- c("gender","gender")
colnames(gender_counts)[1] <- "value"
sample_type_counts$category <- c("sample_type","sample_type")
colnames(sample_type_counts)[1] <- "value"
vital_status_counts$category <- c("vital_status","vital_status")
colnames(vital_status_counts)[1] <- "value"
# coombined 3 tables
combined_counts <-rbind(gender_counts, sample_type_counts, vital_status_counts)
combined_counts <- combined_counts[,c("category","value", "n")]
combined_counts

### Data cleanup

- The newly released gene expression quantification files were generated using STAR workflow, which were written in a different format compared to the previous version using HTSeq workflow. Therefore, we need to wirte our own merge function to merge RNA-seq counts data.

In [ ]:
# Define the function to merge all RNAseq quantification files into one datadrame
merge_rna <-function(metadata, fdir){
    filelist <- list.files(fdir, pattern="*.tsv$", 
                        recursive = TRUE, full.names=TRUE)
    for (i in 1:length(filelist)){
        iname <- basename(filelist[i])
        isamplename <- metadata[metadata$file_name==iname, "sample"]
        idf <- read.csv(filelist[i], sep="\t", skip=1, header=TRUE)
        # remove first 4 rows
        remove <- 1:4
        idf_subset <- idf[-remove, c("gene_id","unstranded")]
        rm(idf)
        names(idf_subset)[2] <- isamplename
        #print(dim(idf_subset))
        if (i==1){
            combined_df <- idf_subset
            rm(idf_subset)
        } else {
            combined_df <- merge(combined_df, idf_subset, by.x='gene_id', by.y="gene_id", all=TRUE)
            rm(idf_subset)
        }
    }
    # remove certain gene ids
    combined_df <- combined_df[!(grepl("PAR_Y", combined_df$gene_id, fixed=TRUE)),]
    # modify gene_id
    combined_df$gene_id <- sapply(strsplit(combined_df$gene_id,"\\."), `[`, 1)
    # use gene_id as row names and remove gene_id column
    rownames(combined_df) <- combined_df$gene_id
    combined_df <- combined_df[,-which(names(combined_df) %in% c("gene_id"))]
    return(combined_df)
}

rnaCounts <-  merge_rna(metaMatrix.RNA, "TCGA-CHOL_0406/RNAseq")
rnaCounts[1:5,]

- Next block shows the number of genes in this dataframe

In [ ]:
# show the number of genes in the rnaCounts dataset
dim(rnaCounts)

- A method of `gdcVoomNormalization()` performs TMM normalization using `edgeR package` (Robinson, McCarthy, and Smyth 2010) and further transforms the data by the voom method using `limma package` (Ritchie et al. 2015). We can transformt the expression counts using this method.

In [ ]:
# Normalization of RNAseq data 
rnaExpr <- gdcVoomNormalization(counts = rnaCounts, filter = FALSE)

## Differential expression analysis

Here, we use RNA-seq quantification data as an example to perform `differential gene expression analysis (DE)`using GDCRNATools package. The method we're using here is `DESeq2`, which uses the raw counts and models the normalization inside the Generalized Linear Model (GLM). Users have option to choose other DE analysis tools, including `edgeR and limma`. 

In [ ]:
DEGAll_CHOL<- gdcDEAnalysis(counts = rnaCounts, 
                        group      = metaMatrix.RNA$sample_type, 
                        comparison = 'PrimaryTumor-SolidTissueNormal', 
                        method     = 'DESeq2',
                        filter=TRUE)

- The next block shows first five lines of DE analysis output

In [ ]:
DEGAll_CHOL[1:5,]

### DE analysis visualization


- The `Volcano plot` is a scatterplot to visualize statistical significance (P value) versus magnitude of change (fold change). It allows quick identification of genes which shows large fold changes between groups and are also statistically significant.

In [ ]:
gdcVolcanoPlot(DEGAll_CHOL)

- The `Barplot` shows the composition of differntially expressed genes based on gene type.

In [ ]:
gdcBarPlot(deg = DEGAll_CHOL, angle = 45, data.type = 'RNAseq')

- Next step, we can filter out all the significant differentially expressed genes based on gene type.
- Filtering criteria used are `fc = 2, pval = 0.01`.

In [ ]:
# all DE genes passed filtering
deALL_CHOL <- gdcDEReport(deg = DEGAll_CHOL, gene.type = 'all')

# DE long-noncoding
deLNC_CHOL <- gdcDEReport(deg = DEGAll_CHOL, gene.type = 'long_non_coding')

# DE protein coding genes
dePC_CHOL <- gdcDEReport(deg = DEGAll_CHOL, gene.type = 'protein_coding')

- A total of 361 protein coding genes were found differentially expressed between `PrimaryTumor` and `SolidTissueNormal` with statistical significance.

In [ ]:
dim(dePC_CHOL)
dim(deLNC_CHOL)

- Visualize a correlation between two genes in the `dePC_CHOL dataset`. Both genes were identified as `up-regulated` in the tumor tissue.

In [ ]:
gdcCorPlot(gene1    = "ENSG00000103569", 
           gene2    = "ENSG00000118271", 
           rna.expr = rnaExpr, 
           metadata = metaMatrix.RNA)

## Functional enrichment analysis

The method of `gdcEnrichAnalysis` is able to take the output of `gdcDEReport` as input and perform gene enrichment analysis.

In [ ]:
enrichOutput <- gdcEnrichAnalysis(gene = rownames(deALL_CHOL), simplify = TRUE)

In [ ]:
enrichOutput[1:3,]

- Visualize the gene enrichment results as `barplot`

In [ ]:
# adjust the plot size
options(repr.plot.width = 12, repr.plot.height = 8, repr.plot.res = 100)
# plot result of gene enrichment analysis using KEGG category. 
gdcEnrichPlot(enrichOutput, type='bar', category='KEGG', num.terms = 20)

In [ ]:
gdcEnrichPlot(enrichOutput, type='bar', category='GO_BP', num.terms = 20)

- Visualize the gene enrichment results as `bubble plot`

In [ ]:
# alternatively, user can visualize the gene enrichment results in bubble plot
options(repr.plot.width = 12, repr.plot.height = 8, repr.plot.res = 100)
gdcEnrichPlot(enrichOutput, type='bubble', category='KEGG', num.terms = 20)

- The next block shows the top 5 of gene enrichment analysis results based on `gene ontology biological process` (GO_BP) 
- We observed a lot of differentially expressed gene involved in `fatty acid metabolic process`, `small molecule catabolic process`, `organic acid biosynthetic process`. Liver is the central organ for fatty acid metabolism. 

In [ ]:
enrichOutput[grep("GO_BP", enrichOutput$Category),] %>% arrange( desc(Counts)) %>% head(5)

- The next block shows the top 5 of gene enrichment analysis results based on `Kyoto Encyclopedia of Genes and Genomes` (KEGG)
- We observed the a lot of differentially expressed genes were enriched in `Complement and coagulation cascades`, `Drug metabolism`, `PPAR signaling pathway`, and `Cholesterol metabolism`.

In [ ]:
enrichOutput[grep("KEGG", enrichOutput$Category),] %>% arrange(desc(Counts)) %>% head(5)

- In this gene enrichment output, few differentially expressed genes, such as apolipoprotein A2 (APOA2), apolipoprotein B (APOB), fibrinogen alpha chain (FGA), fibrinogen gamma chain (FGG), were previsouly reported to be nefatively correlated with the tumor stage of CCA patients. ([Li et al. 2019](https://pubmed.ncbi.nlm.nih.gov/31545466/))

## Univariate survival analysis

- In this analysis, we uses Kaplan Meier(KM) method based on the `survival` R package. KM analysis divides patients into high-expression and low-expression groups by a user-defined threshold. We used `median` as the threshold in this analysis.
- The next block shows how to conduct KM survival analysis using differentially expressed gene list, metadata, and normalized expression quantification dataframe.

In [ ]:
survOutput <- gdcSurvivalAnalysis(gene     = rownames(deALL_CHOL), 
                                  method   = 'KM', 
                                  rna.expr = rnaExpr, 
                                  metadata = metaMatrix.RNA, 
                                  sep      = 'median')

- Inspect the first five lines of survival analysis

In [ ]:
# Sort the output of univariate survival analysis based on pvalue
survOutput$pValue <- as.numeric(survOutput$pValue)
sorted_survOutput<- survOutput[order(survOutput$pValue),]
sorted_survOutput[1:5,]

- The next block shows the KM survival plot using the top gene (ENSG00000123358) from survival analysis output.

In [ ]:
# pick the most significant gene ENSG00000123358 for KM visualization
gdcKMPlot(gene     = 'ENSG00000123358',
          rna.expr = rnaExpr,
          metadata = metaMatrix.RNA,
          sep      = 'median')